# 阿里天池新人赛 工业蒸汽量预测

#### 赛题来自[工业蒸汽量预测](https://tianchi.aliyun.com/competition/entrance/231693/introduction?spm=5176.12281925.0.0.c8667137BbWnWx)  

大致背景如下，工业上蒸汽产量主要受到 燃烧给量，一二次风，引风，返料风，给水水量；以及锅炉的工况，比如锅炉床温、床压，炉膛温度、压力，过热器的温度等因素的影响，题目共给出了2个数据集(train.txt and test.txt), 其中含有38种可能影响蒸汽产量的变量数据，对应字段为”V0”-“V37”，其中只有train.txt中包含'target'字段，表示目标变量即蒸汽产量，目标为用已知数据作为训练样本，用训练后的算法模型预测 测试数据中的'target'字段

## Content
* 环境
* 大致思路
* 代码实现
* 提升方向
* 学习资料

### 环境

需要安装一些常用的 python 的包和库，具体名称和安装方法及教程如下<br>
[numpy安装及教程(中文版)](http://www.runoob.com/numpy/numpy-tutorial.html)<br>
[numpy官方文档(英文版)](http://www.numpy.org/)<br>
[pandas安装及教程(中文版)](https://morvanzhou.github.io/tutorials/data-manipulation/np-pd/)<br>
[pandas官方文档(英文版)](https://pandas.pydata.org/)<br>
[seaborn官方文档](https://seaborn.pydata.org/installing.html#installingb)<br>
[matplotlib](https://matplotlib.org/faq/installing_faq.html)<br>
[sklearn安装](https://morvanzhou.github.io/tutorials/machine-learning/sklearn/1-2-install/)
<br><br>
总的来说几条命令<br>
pip3 install numpy<br>
pip3 install pandas<br>
pip3 install seaborn<br>
pip3 install matplotlib<br>
pip3 install scikit-learn

### 大致思路

这是一个典型的预测数值的应用，采用线性回归处理，大体解决流程是<br>


1. 特征选择： 并不是所有特征都是有用的，一些不好的或带有过多outliers的特征会对结果造成比较大的影响，我们可以先做可视化，肉眼观察，去除掉一些不近似符合正态分布的特征数据，之后再移除掉方差过低的一些特征，然后通过计算筛选出与结果相关性较高的前N个特征组成训练数据集；

2. 数据处理： 对数据进行归一化处理，使所有数值在一定小的范围内，这样能够保证算法快速收敛，降低计算成本；

3. 模型选择： 这里算法选择了GradientBoostingRegressor，是一种通过构建多个回归树(Regression Tree)并拟合梯度(近似于残差(residual))的线性回归模型；

4. 模型调优： 通过模型在测试集上的表现调整模型中的参数

### 代码

In [ ]:
# 导入必要的包
import numpy as np
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import sys
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
# 导入数据集
with open('zhengqi_train.txt') as fr:
    data_train = pd.read_csv(fr, sep='\t')
with open('zhengqi_test.txt') as fr_test:
    data_test = pd.read_csv(fr_test, sep='\t')
    
data_train['origin'] = 'train'
data_test['origin'] = 'test'
data_all = pd.concat([data_train, data_test], axis=0)
#data_all.head(5)

In [ ]:
# 可视化
fig = plt.figure(figsize=(6,6))
for column in data_all.columns[0:-2]:
    g = sns.kdeplot(data_all[column][(data_all['origin'] == 'train')], 
                    color = 'Red', shade = True)
    g = sns.kdeplot(data_all[column][(data_all['origin'] == 'test')], 
                    color = 'Blue', shade = True, ax=g)
    g.set_xlabel(column)
    g.set_ylabel('Frequency')
    g = g.legend(['train', 'test'])
    plt.show()

In [ ]:
# 根据观察结果删除部分特征（在最终提交时没有选择手动删除这些特征）
#train = data_train.drop(['V14', 'V17', 'V22', 'V35', 'V5', 'V9', 'origin', 'target'], axis=1)
#y = data_train['target']
#test = data_test.drop(['V14', 'V17', 'V22', 'V35', 'V5', 'V9', 'origin'], axis=1)

train = data_train.drop(['origin', 'target'], axis=1)
test = data_test.drop(['origin'], axis=1)
y = data_train['target']

In [ ]:
# 删除方差过低的特征
from sklearn.feature_selection import VarianceThreshold
th = 0.85
sel = VarianceThreshold(threshold = (th * (1 - th)))
vt = sel.fit(train)

#find feature names
feat_var_threshold = train.columns[vt.variances_ > th * (1 - th)]
train = train[feat_var_threshold]
test = test[feat_var_threshold]

In [ ]:
# 选择相关性最强的前20个特征
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import preprocessing
x_scored = SelectKBest(score_func=f_regression, k='all').fit(train, y)

feature_scoring = pd.DataFrame({
    'feature':train.columns,
    'score':x_scored.scores_
})

K = 20
feat_scored_headnum = feature_scoring.sort_values('score', 
                                                  ascending=False).head(K)['feature']

train_head = train[train.columns[train.columns.isin(feat_scored_headnum)]]
test_head = test[test.columns[test.columns.isin(feat_scored_headnum)]]

train_ = pd.DataFrame(preprocessing.scale(train_head), columns=train_head.columns)
test_ = pd.DataFrame(preprocessing.scale(test_head), columns=test_head.columns)


In [ ]:
# 数据归一化，并把数据集切分为8:2，多数用于训练，少数用于测试
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

train_ = MinMaxScaler().fit_transform(train_)
test_ = MinMaxScaler().fit_transform(test_)


x_train, x_test, y_train, y_test = train_test_split(train_, y, 
                                                    test_size=0.2,
                                                   random_state=100)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
# 导入集成好的模型，不断调参
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

myGBR = GradientBoostingRegressor(loss='huber', alpha=0.9, criterion='friedman_mse', init=None,
                                  learning_rate=0.01, max_depth=10,
                                  max_features='sqrt', max_leaf_nodes=None,
                                  min_impurity_decrease=0.0, min_impurity_split=None,
                                  min_samples_leaf=12, min_samples_split=15,
                                  min_weight_fraction_leaf=0.0, n_estimators=20000,
                                  presort='auto', random_state=10, subsample=0.9, verbose=0,
                                  warm_start=False)

score = make_scorer(mean_squared_error)
scores = cross_val_score(myGBR, x_train, y_train, cv=10, scoring=score)
print("myGBR : {:.4f} ({:.4f})\n".format(scores.mean(), scores.std()))
myGBR.fit(x_train, y_train)
y_pred = myGBR.predict(x_test)
print(mean_squared_error(y_test, y_pred))

In [ ]:
# 用训练好的模型做预测并保存预测结果
result = myGBR.predict(test_)
print(result)
for i in range(0, len(result)):
    result[i] = round(result[i], 3)
result = pd.DataFrame(result)
result.to_csv('result.txt', index=False, header=False)

### 提升方向

这个方法最终线上测评的结果是 0.1374（均方误差），这里只有了一种模型进行预测，可以考虑用多个模型同时预测，然后对结果加权或直接取平均值，特征选择部分还可以考虑更多的方法<br><br>

昨晚尝试的过程中实用了多种模型预测，然后对结果加权拟合的方法，最终线上结果为 0.1337（排名上升了一百多...）

## 学习资料

### 关于线性回归

关于线性回归，简书上有一篇写的不错的介绍[文章](https://www.jianshu.com/p/fb9712b6e5ea)，知乎上线性回归话题下也有不少优质解答，此外youtube上有吴恩达教授(Andrew Ng)的机器学习系列视频，也是从线性回归开始讲起的，[传送门](https://www.youtube.com/watch?v=qBG9aFTFTwc&list=PLOXON7BTL9IW7Ggbc09jLqGmzkwPI4-3V&index=2); 周志华的《机器学习》也有讲到各种算法，[京东链接](https://item.jd.com/11867803.html)；英文书籍暂时还没有发现比较合适的读物，找到后会立即更新

### 关于提到的决策树和GradientBoostingRegressor

关于决策树相关知识力荐 李航老师的《统计学习方法》，[京东链接](https://item.jd.com/12385906.html)，书中详细讲解了机器学习常见的算法理论；英文书籍稍后更新；关于GradientBoostingRegressor模型，华盛顿大学Tianqi Chen的  [PPT](https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf) 以及 Kaggle上一位神仙写的 [文章](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/) 有比较详细的解释

### 关于机器学习库sklearn

这个库中集成了几乎所有的模型、数据处理方式、特征选择方法、模型评估方法，它的官方文档和教程也是目前见过写的最简洁明了的文档之一，[传送门](https://scikit-learn.org/stable/modules/classes.html)

### 关于数据处理和可视化

可视化部分主要用到 seaborn 这个库，科赛网上有一篇不错的介绍和教程，[低维度特征可视化与强特征构造 ](https://www.kesci.com/home/project/59f687e1c5f3f511952baca0)；关于特征和数据处理，美团技术团队有一篇结合实例的分析文章，[机器学习中的数据清洗与特征处理综述](https://tech.meituan.com/2015/02/10/machinelearning-data-feature-process.html);已经他们的另一片文章，[实例讲解机器学习如何解决问题](https://tech.meituan.com/2015/02/05/mt-mlinaction-how-to-ml.html)

最后推荐一个个人网站，上海交大博士张宏伦的个人博客，[生命不息|张宏伦的个人网站](http://zhanghonglun.cn/blog/)，里面有很多不错的文章、教程以及小demo